In [1]:
import os, pickle, gc

import numpy as np
import pandas as pd

from tqdm import tqdm

from collections import defaultdict, Counter
import cudf, itertools

import pandarallel
from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8, progress_bar=True)

# supress warnings
import warnings
warnings.filterwarnings('ignore')

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
class config:
    data_path = '../data/'
    local_validation = True
    validation_path = '../data/local_validation/'
    train_file = 'train.parquet'
    test_file = 'test.parquet'
    test_labels_file = 'test_labels.parquet'
    n_session_samples = 100
    n_most_common = 50
    debug = False

In [3]:
if config.local_validation:
    train = cudf.read_parquet(config.validation_path + config.train_file)
    test = cudf.read_parquet(config.validation_path + config.test_file)
    test_labels = cudf.read_parquet(config.validation_path + config.test_labels_file)
    data = cudf.concat([train, test])
else:
    train = cudf.read_parquet(config.data_path + config.train_file)
    test = cudf.read_parquet(config.data_path + config.test_file)
    data = cudf.concat([train, test])    

In [4]:
if config.debug:
    data = data.sample(frac=0.02, random_state=42)

In [5]:
type_labels = {'clicks':0, 'carts':1, 'orders':2}
type_weight = {0:1, 1:6, 2:3}
version = 1

### Create co-visitation matrix on GPU using CuDF

### Carts Orders Co-visitation Matrix

In [6]:
# This code creates a dictionary mapping each product (aid) to the top 50 most similar products (aid_y) based on the type of product (type_y).
# To do this, it first creates all possible pairs of products for each session. Then it calculates the weight of the pair based on type_y.
# We then calculate the top 50 similar products for each product based on the sum of the weights of all the edges from the same source to target.

data_copy = data.copy()
data_copy = data_copy.set_index('session')
sessions = data_copy.index.unique()

chunk_size = 100_000

tmp = list()
# as we are processing sessions in chunks
# over iterations we can append duplicate aid_x and aid_y pairs
for i in tqdm(range(0, sessions.shape[0], chunk_size)):
    df = data_copy.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index()
    df = df.sort_values(['session','ts'],ascending=[True, False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n', axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs() < 24 * 60 * 60) & (df.aid_x != df.aid_y) ]

    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    df['wgt'] = df.type_y.map(type_weight)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    # aid_x is the source, aid_y is the target
    # calculate the sum of weights based on target type
    # we sum the weights of all the edges from same source to target
    df = df.groupby(['aid_x','aid_y']).wgt.sum()

    tmp.append(df.reset_index())
    
    del df
    gc.collect()

tmp = list(map(lambda x: x.to_pandas(), tmp))
tmp = pd.concat(tmp)
tmp = tmp.groupby(['aid_x','aid_y']).wgt.sum().reset_index()
tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
tmp = tmp.reset_index(drop=True)
tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
# we only select 50 products for each aid_x
tmp = tmp.loc[tmp.n<15].drop('n',axis=1)
# SAVE TO DISK
df = tmp.groupby('aid_x').aid_y.apply(list)
with open(config.data_path + f'top_15_carts_orders_v{version}.pkl', 'wb') as f:
    pickle.dump(df.to_dict(), f)
    
del df, tmp
gc.collect()

100%|██████████████████████████████████████████████| 129/129 [00:52<00:00,  2.47it/s]


0

### Buy2Buy Co-visitation matrix

In [7]:
# %%time
data_copy = data.copy()
data_copy = data_copy.set_index('session')
sessions = data_copy.index.unique()

chunk_size = 100_000

tmp = list()
for i in tqdm(range(0, sessions.shape[0], chunk_size)):
    df = data_copy.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index()
    df = df.loc[(df['type']==1)|(df['type']==2)]

    df = df.sort_values(['session','ts'], ascending=[True, False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df, on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs() < 14 * 24 * 60 * 60) & (df.aid_x != df.aid_y) ]

    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','type_y']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    df['wgt'] = 1
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum()

    tmp.append(df.reset_index())
       
    del df
    gc.collect()

tmp = list(map(lambda x: x.to_pandas(), tmp))
tmp = pd.concat(tmp)
tmp = tmp.groupby(['aid_x','aid_y']).wgt.sum().reset_index()
tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
tmp.drop_duplicates(['aid_x','aid_y'],inplace=True)
# SAVE TOP 15
tmp = tmp.reset_index(drop=True)
tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
tmp = tmp.loc[tmp.n<15].drop('n',axis=1)
# SAVE TO DISK
df = tmp.groupby('aid_x').aid_y.apply(list)
with open(config.data_path + f'top_15_buy2buy_v{version}.pkl', 'wb') as f:
    pickle.dump(df.to_dict(), f)
    
del df, tmp
gc.collect()

100%|██████████████████████████████████████████████| 129/129 [00:34<00:00,  3.78it/s]


0

### Clicks Co-visitation matrix

In [8]:
#%%time
data_copy = data.copy()
data_copy = data_copy.set_index('session')
sessions = data_copy.index.unique()

chunk_size = 100_000

tmp = list()
for i in tqdm(range(0, sessions.shape[0], chunk_size)):
    df = data_copy.loc[sessions[i]:sessions[min(sessions.shape[0]-1, i+chunk_size-1)]].reset_index()
    df = df.sort_values(['session','ts'], ascending=[True, False])

    # USE TAIL OF SESSION
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df = df.loc[df.n<30].drop('n',axis=1)

    # CREATE PAIRS
    df = df.merge(df,on='session')
    df = df.loc[ ((df.ts_x - df.ts_y).abs() < 24 * 60 * 60) & (df.aid_x != df.aid_y) ]

    # ASSIGN WEIGHTS
    df = df[['session', 'aid_x', 'aid_y','ts_x']].drop_duplicates(['session', 'aid_x', 'aid_y'])
    # assign weights based on time
    # 1 + 3*(ts_x - min(ts))/(max(ts)-min(ts))
    # min(ts) = 1659304800
    # max(ts) = 1662328791
    # more recent the click, higher the weight
    # factor can be a hyperparameter - to be finalized
    df['wgt'] = 1 + 3*(df.ts_x - 1659304800)/(1662328791-1659304800)
    df = df[['aid_x','aid_y','wgt']]
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.sum()

    tmp.append(df.reset_index())
    
    del df
    gc.collect()

tmp = list(map(lambda x: x.to_pandas(), tmp))
tmp = pd.concat(tmp)
tmp.fillna(0,inplace=True)
tmp = tmp.groupby(['aid_x','aid_y']).wgt.sum().reset_index()
tmp = tmp.sort_values(['aid_x','wgt'],ascending=[True,False])
tmp.drop_duplicates(['aid_x','aid_y'],inplace=True)
tmp = tmp.reset_index(drop=True)
tmp['n'] = tmp.groupby('aid_x').aid_y.cumcount()
tmp = tmp.loc[tmp.n<20].drop('n',axis=1)
df = tmp.groupby('aid_x').aid_y.apply(list)
with open(config.data_path + f'top_20_clicks_v{version}.pkl', 'wb') as f:
    pickle.dump(df.to_dict(), f)
    
del df, tmp
gc.collect()

100%|██████████████████████████████████████████████| 129/129 [00:55<00:00,  2.33it/s]


0

In [9]:
%%time
top_20_clicks = pd.read_pickle(config.data_path + f'top_20_clicks_v{version}.pkl')
top_15_buys = pd.read_pickle(config.data_path + f'top_15_carts_orders_v{version}.pkl')
top_15_buy2buy = pd.read_pickle(config.data_path + f'top_15_buy2buy_v{version}.pkl')

# TOP CLICKS AND ORDERS IN TEST
top_clicks = test.loc[test['type']==type_labels['clicks'],'aid'].value_counts().index.values[:20]
top_orders = test.loc[test['type']==type_labels['orders'],'aid'].value_counts().index.values[:20]

# print shape of each matrix
print(f'clicks: {len(top_20_clicks)}')
print(f'carts: {len(top_15_buys)}')
print(f'buy2buy: {len(top_15_buy2buy)}')

clicks: 1812132
carts: 1812132
buy2buy: 1055146
CPU times: user 7.25 s, sys: 530 ms, total: 7.78 s
Wall time: 7.79 s


In [10]:
def suggest_clicks(df, top_20_clicks, top_clicks):
    products = df.aid.tolist()
    types = df.type.tolist()
    unique_products = list(dict.fromkeys(products[::-1] ))

    if len(unique_products) >= 20:
        weights = np.logspace(0.1, 1, len(products), base=2, endpoint=True) - 1
        products_tmp = Counter()

        for product, weight, _type in zip(products, weights, types):
            products_tmp[product] += weight * type_weight[_type]
        
        sorted_products = [product for product, _ in products_tmp.most_common(50)]
        return sorted_products

    # check if it is possible to index into dataframe using productid
    products_1 = list(itertools.chain(*[top_20_clicks[product] \
                    for product in unique_products if product in top_20_clicks]))
    top_products_1 = [product for product, _ in Counter(products_1).most_common(50) \
                    if product not in unique_products]
    result = unique_products + top_products_1[:20 - len(unique_products)]
    return result + list(top_clicks[:20 - len(result)])

def suggest_buys(df, top_15_buy2buy, top_15_buys, top_orders):
    products = df.aid.tolist()
    types = df.type.tolist()
    # filter df for type 1 and 2
    unique_products = list(dict.fromkeys(products[::-1] ))
    df = df.loc[(df['type']==1)|(df['type']==2)]
    unique_buys = list(dict.fromkeys(df.aid.tolist()[::-1]))

    if len(unique_products) >= 20:
        weights = np.logspace(0.5, 1, len(products), base=2, endpoint=True) - 1
        products_tmp = Counter()
        for product, weight, _type in zip(products, weights, types):
            products_tmp[product] += weight * type_weight[_type]
        products_1 = list(itertools.chain(*[top_15_buy2buy.get(product, []) \
                        for product in unique_buys if product in top_15_buy2buy]))
        for product in products_1: products_tmp[product] += 0.1
        sorted_products = [product for product, _ in products_tmp.most_common(50)]
        return sorted_products

    products_1 = list(itertools.chain(*[top_15_buys.get(product, []) \
                        for product in unique_products if product in top_15_buys]))
    products_2 = list(itertools.chain(*[top_15_buy2buy.get(product, []) \
                        for product in unique_buys if product in top_15_buy2buy]))
    top_products = [product for product, _ in Counter(products_1 + products_2).most_common(50) \
                    if product not in unique_products]
    result = unique_products + top_products[:20 - len(unique_products)]
    return result + list(top_orders[:20 - len(result)])


In [11]:
%%time
# if not config.local_validation:
test = test.to_pandas()

tqdm.pandas()

pred_df_clicks = test.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(
    lambda x: suggest_clicks(x, top_20_clicks, top_clicks)
)

100%|███████████████████████████████████| 1801251/1801251 [02:26<00:00, 12305.98it/s]

CPU times: user 2min 28s, sys: 10.1 s, total: 2min 38s
Wall time: 2min 27s


In [12]:
pred_df_buys = test.sort_values(["session", "ts"]).groupby(["session"]).progress_apply(
    lambda x: suggest_buys(x, top_15_buy2buy, top_15_buys, top_orders)
)

100%|████████████████████████████████████| 1801251/1801251 [11:56<00:00, 2514.67it/s]


In [13]:
clicks_pred_df = pd.DataFrame(pred_df_clicks.add_suffix("_clicks"), columns=["labels"]).reset_index()
orders_pred_df = pd.DataFrame(pred_df_buys.add_suffix("_orders"), columns=["labels"]).reset_index()
carts_pred_df = pd.DataFrame(pred_df_buys.add_suffix("_carts"), columns=["labels"]).reset_index()

In [14]:
pred_df = pd.concat([clicks_pred_df, orders_pred_df, carts_pred_df])
# pred_df = clicks_pred_df
pred_df.columns = ["session_type", "labels"]
pred_df["labels"] = pred_df.labels.progress_apply(lambda x: " ".join(map(str,x)))

100%|███████████████████████████████████| 5403753/5403753 [04:35<00:00, 19646.19it/s]


In [15]:
%%time
# COMPUTE METRIC

if config.local_validation:
    score = 0
    weights = {'clicks': 0.10, 'carts': 0.30, 'orders': 0.60}
    for t in ['clicks','carts','orders']:
    # for t in ['clicks']:
        sub = pred_df.loc[pred_df.session_type.str.contains(t)].copy()
        sub['session'] = sub.session_type.apply(lambda x: int(x.split('_')[0]))
        sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')[:20]])
        test_labels = pd.read_parquet('../data/local_validation/test_labels.parquet')
        test_labels = test_labels.loc[test_labels['type']==t]
        test_labels = test_labels.merge(sub, how='left', on=['session'])
        test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1)
        test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,20)
        recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
        score += weights[t]*recall
        print(f'{t} recall =',recall)
        
    print('=============')
    print('Overall Recall =',score)
    print('=============')

clicks recall = 0.5255722760140219
carts recall = 0.40959301417910704
orders recall = 0.6489245235443005
Overall Recall = 0.5647898459817146
CPU times: user 42.6 s, sys: 982 ms, total: 43.6 s
Wall time: 43.3 s


In [16]:
pred_df.to_csv(config.data_path + 'submission.csv', index=False)
pred_df.head()

,session_type,labels
0,11098528_clicks,11830 588923 1732105 571762 884502 1157882 876...
1,11098529_clicks,1105029 459126 1339838 1544564 217742 1694360 ...
2,11098530_clicks,409236 264500 1603001 963957 254154 583026 167...
3,11098531_clicks,396199 1271998 452188 1728212 1365569 624163 1...
4,11098532_clicks,876469 7651 108125 1202618 1159379 77906 17040...
